# First 2 XGBOOST APPROACH WITH DATA RESCALED
# BUT NOT SMOOTHED

This is due to better empirical performences of the sets and visually. 
But using rolling3 smoothing dataset, and Optimizing better, we believe we would get better results.  

In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
import pandas.api.types as ptypes
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt

train = pd.read_csv("/Users/pacohoche/Desktop/Hackathon Git/HackatonInnovAItive/data/processed/train_1min.csv")
test = pd.read_csv("/Users/pacohoche/Desktop/Hackathon Git/HackatonInnovAItive/data/raw/test.csv")
# SORRY, BUT I CANNOT UPLOAD THE DATA FILES TO TEST THE CODE.


def make_features(df):
    df = df.copy()

    # ensure datetime is parsed
    if not ptypes.is_datetime64_any_dtype(df["datetime"]):
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True, errors="coerce")

    df["hour"] = df["datetime"].dt.hour
    df["minute"] = df["datetime"].dt.minute
    df["dayofweek"] = df["datetime"].dt.dayofweek

    return df

def make_window_features(df, window=10, is_train=True):
    """
    Creates lag features for a sliding window of size `window`.
    - If `home_id` exists: lags are computed *within each home*.
    - Otherwise: lags are computed globally.
    Names and dimensions are unchanged.
    """
    df = df.copy()

    if "home_id" in df.columns:
        # sort by home and time, like before (for both train and test)
        df = df.sort_values(["home_id", "datetime"])
        group_power = df.groupby("home_id")["power"]

        for lag in range(1, window + 1):
            df[f"lag_{lag}"] = group_power.shift(lag)
    else:
        # fallback: no home_id → treat as a single sequence (like your old test path)
        df = df.sort_values("datetime")
        for lag in range(1, window + 1):
            df[f"lag_{lag}"] = df["power"].shift(lag)

    # Fill initial NaNs with 0 (same as before)
    lag_cols = [f"lag_{lag}" for lag in range(1, window + 1)]
    df[lag_cols] = df[lag_cols].fillna(0)

    return df


train = make_features(train)
test = make_features(test)

train = make_window_features(train, window=20, is_train=True)
test = make_window_features(test, window=20, is_train=False)


FEATURES = ["power", "hour", "minute", "dayofweek"] + [f"lag_{lag}" for lag in range(1, 21)]
X = train[FEATURES]
y = train["fridge"]
X_test = test[FEATURES]


n_groups = train["home_id"].nunique()
print("Number of distinct homes in train:", n_groups)

models = []

if n_groups >= 2:
    n_splits = min(10, n_groups)
    print("Using GroupKFold with n_splits =", n_splits)

    gkf = GroupKFold(n_splits=n_splits)
    preds_oof = np.zeros(len(train))

    for tr_idx, val_idx in gkf.split(X, y, groups=train["home_id"]):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = XGBRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            objective="reg:squarederror",
            random_state=42,
        )

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            verbose=False,
        )

        preds_oof[val_idx] = model.predict(X_val)
        models.append(model)

    print("CV RMSE:", np.sqrt(mean_squared_error(y, preds_oof)))
    print("CV MAE:", mean_absolute_error(y, preds_oof))
else:
    # These are not enough groups for GroupKFold, technically we wouldn't be able to do CV.
    # in fact we just do the regression, but since we talked about replicability. 
    print("Only one group found, training on full data without CV.")

    model = XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )
    model.fit(X, y)
    models.append(model)

train_viz = train.copy()
train_viz["pred_fridge"] = preds_oof

# If we have home_id, plot one time-series per house
if "home_id" in train_viz.columns:
    homes = sorted(train_viz["home_id"].unique())
else:
    homes = [None]

for home in homes:
    if home is not None:
        df_plot = train_viz[train_viz["home_id"] == home].copy()
        title = f"Home {home} — fridge: true vs predicted"
    else:
        df_plot = train_viz.copy()
        title = "Fridge — true vs predicted"

    # Sort by time so the curves make sense
    df_plot = df_plot.sort_values("datetime")

    plt.figure(figsize=(12, 4))
    plt.plot(df_plot["datetime"], df_plot["fridge"], label="True")
    plt.plot(df_plot["datetime"], df_plot["pred_fridge"], label="Predicted", alpha=0.7)
    plt.title(title)
    plt.xlabel("Datetime")
    plt.ylabel("Fridge power")
    plt.legend()
    plt.tight_layout()
    plt.show()


# Predictions. 
test_preds = np.mean([m.predict(X_test) for m in models], axis=0)

# Submission Files 
submission = pd.DataFrame({
    "id": test["id"],   # from your test.csv
    "fridge": test_preds
})

submission.to_csv("submission_Final.csv", index=False)
submission.head()


Number of distinct homes in train: 4
Using GroupKFold with n_splits = 4


XGBoostError: [10:49:08] /Users/runner/miniforge3/conda-bld/xgboost-split_1762060257953/work/src/data/data.cc:565: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000162c84654 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000162e125e0 xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const*, xgboost::StringView, xgboost::Json) + 3260
  [bt] (2) 3   libxgboost.dylib                    0x0000000162e1180c xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 620
  [bt] (3) 4   libxgboost.dylib                    0x0000000162c9b28c XGDMatrixSetInfoFromInterface + 236
  [bt] (4) 5   libffi.8.dylib                      0x00000001021f8050 ffi_call_SYSV + 80
  [bt] (5) 6   libffi.8.dylib                      0x00000001021f589c ffi_call_int + 1444
  [bt] (6) 7   _ctypes.cpython-312-darwin.so       0x0

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pandas.api.types as ptypes


train = pd.read_csv("train_1min.csv")
test = pd.read_csv("test.csv")

# Remove rows with missing target or power
train = train.dropna(subset=["power", "fridge"])


def make_features(df):
    df = df.copy()

    # Ensure datetime is parsed
    if not ptypes.is_datetime64_any_dtype(df["datetime"]):
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True, errors="coerce")

    df["hour"] = df["datetime"].dt.hour
    df["minute"] = df["datetime"].dt.minute
    df["dayofweek"] = df["datetime"].dt.dayofweek
    df["second"] = df["datetime"].dt.second

    # Is this a quarter-hour reading (00, 15, 30, 45 and second == 0)?
    df["is_quarter"] = ((df["minute"] % 15 == 0) & (df["second"] == 0)).astype(int)

    return df


def add_sequence_features_train(df):
    df = df.sort_values(["home_id", "datetime"]).copy()

    # previous power within each home
    df["prev_power"] = df.groupby("home_id")["power"].shift(1)

    # difference to previous measurement
    df["delta_power"] = df["power"] - df["prev_power"]

    # Fill NaNs for the first row of each group
    df[["prev_power", "delta_power"]] = df[["prev_power", "delta_power"]].fillna(0)

    return df


def add_sequence_features_test(df):
    # test is a single household → treat as one sequence
    df = df.sort_values("datetime").copy()

    df["prev_power"] = df["power"].shift(1)
    df["delta_power"] = df["power"] - df["prev_power"]
    df[["prev_power", "delta_power"]] = df[["prev_power", "delta_power"]].fillna(0)

    return df


# Apply feature engineering
train = make_features(train)
test = make_features(test)

train = add_sequence_features_train(train)
test = add_sequence_features_test(test)

# Features

FEATURES = [
    "power",
    "hour",
    "minute",
    "dayofweek",
    "is_quarter",
    "prev_power",
    "delta_power",
]

X = train[FEATURES]
y = train["fridge"]
X_test = test[FEATURES]


n_groups = train["home_id"].nunique()
print("Number of distinct homes in train:", n_groups)

models = []  # will store (model, scaler)

if n_groups >= 2:
    n_splits = min(5, n_groups)
    print("Using GroupKFold with n_splits =", n_splits)

    gkf = GroupKFold(n_splits=n_splits)
    preds_oof = np.zeros(len(train))

    for tr_idx, val_idx in gkf.split(X, y, groups=train["home_id"]):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        # --- Scaling per fold ---
        scaler = StandardScaler()
        X_tr_scaled = scaler.fit_transform(X_tr)
        X_val_scaled = scaler.transform(X_val)

        model = XGBRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            objective="reg:squarederror",
            random_state=42,
        )

        model.fit(
            X_tr_scaled, y_tr,
            eval_set=[(X_val_scaled, y_val)],
            verbose=False,
        )

        preds_oof[val_idx] = model.predict(X_val_scaled)
        models.append((model, scaler))  # store model + its scaler

    print("CV RMSE:", np.sqrt(mean_squared_error(y, preds_oof)))
    print("CV MAE:", mean_absolute_error(y, preds_oof))
else:
    # only one home_id available → just train one model on all data
    print("Only one group found, training on full data without CV.")

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )
    model.fit(X_scaled, y)
    models.append((model, scaler))



# Plots 

train_viz = train.copy()
train_viz["pred_fridge"] = preds_oof

# If we have home_id, plot one time-series per house
if "home_id" in train_viz.columns:
    homes = sorted(train_viz["home_id"].unique())
else:
    homes = [None]

for home in homes:
    if home is not None:
        df_plot = train_viz[train_viz["home_id"] == home].copy()
        title = f"Home {home} — fridge: true vs predicted"
    else:
        df_plot = train_viz.copy()
        title = "Fridge — true vs predicted"

    # Sort by time so the curves make sense
    df_plot = df_plot.sort_values("datetime")

    plt.figure(figsize=(12, 4))
    plt.plot(df_plot["datetime"], df_plot["fridge"], label="True")
    plt.plot(df_plot["datetime"], df_plot["pred_fridge"], label="Predicted", alpha=0.7)
    plt.title(title)
    plt.xlabel("Datetime")
    plt.ylabel("Fridge power")
    plt.legend()
    plt.tight_layout()
    plt.show()


test_preds = np.mean(
    [m.predict(s.transform(X_test)) for (m, s) in models],
    axis=0
)


submission = pd.DataFrame({
    "id": test["id"],
    "fridge": test_preds
})

submission.to_csv("submission.csv", index=False)
submission.head()


## Comments on the plots

Even tho we have a worst MAE on the CV, we have less overfitting on the plot. 
We can see mostly when the total_power is 0, in the previous plots it was still having the fridge in activity, now it's better. 
This is trying with the data not smoothed, so the train_1min.csv file 